In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [6]:
!pip install tensorflow

    100% |████████████████████████████████| 49.1MB 820kB/s 
    100% |████████████████████████████████| 92kB 15.7MB/s 
    100% |████████████████████████████████| 3.1MB 1.9MB/s 
    100% |████████████████████████████████| 6.4MB 7.4MB/s 
    100% |████████████████████████████████| 12.1MB 5.3MB/s 
    100% |████████████████████████████████| 8.8MB 7.0MB/s 
    100% |████████████████████████████████| 81kB 20.7MB/s 
    100% |████████████████████████████████| 327kB 16.4MB/s 
    100% |████████████████████████████████| 890kB 18.9MB/s 
  Running setup.py bdist_wheel for gast ... - \ done
  Stored in directory: /home/jovyan/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for termcolor ... - \ done
  Stored in directory: /home/jovyan/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for absl-py ... - \ | done
  Stored in directory: /home/jovyan/.cache/pip/wheels/

In [10]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [2]:
data_silver=pd.read_csv("Data_silver.csv")
data_gold=pd.read_csv("Data_gold.csv")
data_plat=pd.read_csv("Data_Plat.csv")
data_diamond=pd.read_csv("Data_diamond.csv")

In [137]:
data=data_silver.append([data_gold,data_plat,data_diamond])
X=np.array(data.values[:,7:],'double')
Y=np.array(data.values[:,1],'double')

where_are_NaNs = np.isnan(X)
X[where_are_NaNs] = 0

Y[Y==100]=1
Y[Y==200]=0

Y1=Y
Y = np.array([Y, -(Y-1)]).T

XX, X_TEST, YY, Y_TEST = train_test_split(X, Y, test_size=0.1, random_state=42)

In [138]:
y=YY
y[:,0]

array([ 0.,  1.,  0., ...,  1.,  0.,  0.])

In [72]:
XX_10=XX[:,list(range(0,20))+list(range(58,63))]
X_TEST_10=X_TEST[:,list(range(0,20))+list(range(58,63))]



In [93]:
X_scaler=StandardScaler()
X_scaler.fit(XX)
XX_scaled=X_scaler.transform(XX)   
X_TEST_scaled=X_scaler.transform(X_TEST)

In [74]:
from sklearn.model_selection import StratifiedKFold, train_test_split

In [70]:
?StratifiedKFold.split

## Full

In [108]:
skf = StratifiedKFold(n_splits=5)
X=XX_scaled
y=YY
splits_training_auc =[]
splits_test_auc =[]

lr_settings=[0.1,0.05,0.01,0.005,0.001]
n1_settings=[5,10,15,20,25,30]
n2_settings=[5,10,15,20,25,30]

#neighbors_settings=range(1,50)

#train_auc_3d_pre=np.zeros((5,5,6,6))
test_accuracy_3d=np.zeros((5,5,6,6))

In [109]:
m=0
for train_index, test_index in skf.split(X, y[:,0]):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index] 
    training_epochs = 100
    batch_size = 100
    n_input = 63 # Number of feature
    n_classes = 2 # Number of classes to predict
    j=0
    for lr in lr_settings:
        learning_rate = lr
        k=0
        for n1 in n1_settings:
            n_hidden_1=n1
            l=0
            for n2 in n2_settings:
                n_hidden_2 = n2

                # tf Graph input
                x = tf.placeholder("float", [None, n_input])
                y = tf.placeholder("float", [None, n_classes])

                # Create model
                def multilayer_perceptron(x, weights, biases):
                    # Hidden layer with RELU activation
                    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
                    layer_1 = tf.nn.relu(layer_1)
                    # Hidden layer with RELU activation
                    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
                    layer_2 = tf.nn.relu(layer_2)
                    # Output layer with linear activation
                    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
                    return out_layer

                # Store layers weight & bias
                weights = {
                    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
                    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
                    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
                }

                biases = {
                    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
                    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
                    'out': tf.Variable(tf.random_normal([n_classes]))
                }

                # Construct model
                pred = multilayer_perceptron(x, weights, biases)

                # Define loss and optimizer
                cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
                optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

                # Initializing the variables
                init = tf.global_variables_initializer()
                # Launch the graph

                with tf.Session() as sess:
                    sess.run(init)
                    # Training cycle
                    for epoch in range(training_epochs):
                        avg_cost = 0.
                        total_batch = int(len(X)/batch_size)
                        X_batches = np.array_split(X_train, total_batch)
                        Y_batches = np.array_split(y_train, total_batch)
                        # Loop over all batches
                        for i in range(total_batch):
                            batch_x, batch_y = X_batches[i], Y_batches[i]
                            # Run optimization op (backprop) and cost op (to get loss value)
                            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                                          y: batch_y})
                            # Compute average loss
                            avg_cost += c / total_batch
                        # Display logs per epoch step
                        #if epoch % display_step == 0:
                        #    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
                    #print("Optimization Finished!")

                    # Test model
                    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
                    # Calculate accuracy
                    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
                    print(m,j,k,l)
                    test_accuracy_3d[m,j,k,l]=accuracy.eval({x: X_test, y: y_test})
                    #global result 
                    #result = tf.argmax(pred, 1).eval({x: X_TEST_scaled, y: Y_TEST})
                l=l+1
            k=k+1
        j=j+1
    m=m+1

0 0 0 0
0 0 0 1
0 0 0 2
0 0 0 3
0 0 0 4
0 0 0 5
0 0 1 0
0 0 1 1
0 0 1 2
0 0 1 3
0 0 1 4
0 0 1 5
0 0 2 0
0 0 2 1
0 0 2 2
0 0 2 3
0 0 2 4
0 0 2 5
0 0 3 0
0 0 3 1
0 0 3 2
0 0 3 3
0 0 3 4
0 0 3 5
0 0 4 0
0 0 4 1
0 0 4 2
0 0 4 3
0 0 4 4
0 0 4 5
0 0 5 0
0 0 5 1
0 0 5 2
0 0 5 3
0 0 5 4
0 0 5 5
0 1 0 0
0 1 0 1
0 1 0 2
0 1 0 3
0 1 0 4
0 1 0 5
0 1 1 0
0 1 1 1
0 1 1 2
0 1 1 3
0 1 1 4
0 1 1 5
0 1 2 0
0 1 2 1
0 1 2 2
0 1 2 3
0 1 2 4
0 1 2 5
0 1 3 0
0 1 3 1
0 1 3 2
0 1 3 3
0 1 3 4
0 1 3 5
0 1 4 0
0 1 4 1
0 1 4 2
0 1 4 3
0 1 4 4
0 1 4 5
0 1 5 0
0 1 5 1
0 1 5 2
0 1 5 3
0 1 5 4
0 1 5 5
0 2 0 0
0 2 0 1
0 2 0 2
0 2 0 3
0 2 0 4
0 2 0 5
0 2 1 0
0 2 1 1
0 2 1 2
0 2 1 3
0 2 1 4
0 2 1 5
0 2 2 0
0 2 2 1
0 2 2 2
0 2 2 3
0 2 2 4
0 2 2 5
0 2 3 0
0 2 3 1
0 2 3 2
0 2 3 3
0 2 3 4
0 2 3 5
0 2 4 0
0 2 4 1
0 2 4 2
0 2 4 3
0 2 4 4
0 2 4 5
0 2 5 0
0 2 5 1
0 2 5 2
0 2 5 3
0 2 5 4
0 2 5 5
0 3 0 0
0 3 0 1
0 3 0 2
0 3 0 3
0 3 0 4
0 3 0 5
0 3 1 0
0 3 1 1
0 3 1 2
0 3 1 3
0 3 1 4
0 3 1 5
0 3 2 0
0 3 2 1
0 3 2 2
0 3 2 3
0 3 2 4


ValueError: Shape must be rank 1 but is rank 2 for 'strided_slice_1' (op: 'StridedSlice') with input shapes: [?,2], [1,2512], [1,2512], [1].

In [113]:
full_test=test_accuracy_3d[0]

In [146]:
full_test


array([[[ 0.97770703,  0.9824841 ,  0.98407644,  0.94108278,  0.9824841 ,
          0.97611463],
        [ 0.90605098,  0.92993629,  0.97292995,  0.94426751,  0.97452229,
          0.97452229],
        [ 0.96496814,  0.96337581,  0.88375795,  0.97292995,  0.97292995,
          0.97452229],
        [ 0.6687898 ,  0.97611463,  0.96974522,  0.97133756,  0.97929937,
          0.96974522],
        [ 0.97452229,  0.67993629,  0.91242039,  0.97611463,  0.97929937,
          0.97611463],
        [ 0.55254775,  0.79936308,  0.97452229,  0.70700639,  0.96815288,
          0.95700639]],

       [[ 0.97452229,  0.97133756,  0.97452229,  0.98726118,  0.97452229,
          0.97929937],
        [ 0.97292995,  0.98407644,  0.96656048,  0.97292995,  0.96178347,
          0.96496814],
        [ 0.97452229,  0.96974522,  0.97770703,  0.95859873,  0.96974522,
          0.96974522],
        [ 0.9808917 ,  0.96974522,  0.97611463,  0.97452229,  0.97452229,
          0.96496814],
        [ 0.97452229,  0.968

In [115]:
full_test.shape

(5, 6, 6)

In [119]:
#lr setting
np.argmax(np.mean(full_test,axis=(1,2)))

1

In [ ]:
lr=0.05

In [121]:
#n1 settings
np.argmax(np.mean(full_test,axis=(0,2)))

array([ 0.96358811,  0.95504247,  0.95642251,  0.94957537,  0.94994692,
        0.93014862])

In [ ]:
n1 settings 5

In [122]:
#n2 settings
np.argmax(np.mean(full_test,axis=(0,1)))

4

In [ ]:
n2 25

## 10

In [131]:
XX_10=XX[:,list(range(0,20))+list(range(58,63))]
X_TEST_10=X_TEST[:,list(range(0,20))+list(range(58,63))]

In [140]:
X_scaler=StandardScaler()
X_scaler.fit(XX_10)
XX_scaled=X_scaler.transform(XX_10)   
X_TEST_scaled=X_scaler.transform(X_TEST_10)

In [139]:
skf = StratifiedKFold(n_splits=5)
X=XX_scaled
y=YY
splits_training_auc =[]
splits_test_auc =[]

lr_settings=[0.05,0.01,0.005,0.001]
n1_settings=[5,10,15,20]
n2_settings=[5,10,15,20]

#neighbors_settings=range(1,50)

#train_auc_3d_pre=np.zeros((5,5,6,6))
test_accuracy_3d_10=np.zeros((5,4,4,4))

In [141]:
data=data_silver.append([data_gold,data_plat,data_diamond])
X=np.array(data.values[:,7:],'double')
Y=np.array(data.values[:,1],'double')

where_are_NaNs = np.isnan(X)
X[where_are_NaNs] = 0

Y[Y==100]=1
Y[Y==200]=0

Y1=Y
Y = np.array([Y, -(Y-1)]).T

XX, X_TEST, YY, Y_TEST = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
training_epochs = 100
batch_size = 100
n_input = 25 # Number of feature
n_classes = 2 # Number of classes to predict
learning_rate = 0.01

n_hidden_1=n1
n_hidden_2 = n2

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()
# Launch the graph

with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X)/batch_size)
        X_batches = np.array_split(XX_scaled, total_batch)
        Y_batches = np.array_split(YY, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        #if epoch % display_step == 0:
        #    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    #print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(m,j,k,l)
    test_accuracy_3d_10[m,j,k,l]=accuracy.eval({x: X_TEST, y: Y_TEST})
    #global result 
    #result = tf.argmax(pred, 1).eval({x: X_TEST_scaled, y: Y_TEST})


In [130]:
test_accuracy_3d_10

array([[[[ 0.55254775,  0.6687898 ,  0.54936308,  0.67197454,  0.66560507,
           0.69267517],
         [ 0.67834395,  0.55573249,  0.63535035,  0.66242039,  0.66401273,
           0.63216561],
         [ 0.66719747,  0.64490443,  0.66719747,  0.67038214,  0.63694268,
           0.65605098],
         [ 0.63375795,  0.56050956,  0.63057327,  0.60350317,  0.61624205,
           0.56847131],
         [ 0.55414015,  0.63535035,  0.64968151,  0.60828024,  0.55732483,
           0.65764332],
         [ 0.55254775,  0.55095541,  0.58280253,  0.5812102 ,  0.56369424,
           0.54936308]],

        [[ 0.69745225,  0.67197454,  0.68312103,  0.6878981 ,  0.68471336,
           0.67356688],
         [ 0.67197454,  0.65127391,  0.66242039,  0.65764332,  0.64968151,
           0.66082805],
         [ 0.66082805,  0.6433121 ,  0.64012736,  0.64490443,  0.65286624,
           0.66560507],
         [ 0.66242039,  0.68471336,  0.64171976,  0.62898088,  0.66082805,
           0.64808917],
        